In [ ]:
# for dev purpose - get username
import os
USERNAME = os.getenv('USERNAME').lower()
new_pull = True

In [ ]:
# import python packages
import sys
if sys.version_info[1]==12:
    os.add_dll_directory(r'C:/Program Files/Python/312/Lib/site-packages/clidriver/bin')
else:
    pass
import pandas as pd
import numpy as np
import datetime as dt
import time
import warnings
from snp_query_box import DsnpHelperFunction, snpQueries
from snp_query_box.dsnp_transform import clinicalConditionsTransform

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
print(f"User: {USERNAME} Started creating member_phone_info_df in the shared storage")

In [ ]:
# set the starting time
start_time = time.time()

In [ ]:
# calculate the dates
today = dt.datetime.today()
today_str = today.strftime('%Y-%m-%d')
last_date_of_last_month_str = DsnpHelperFunction.last_date_of_last_month(today_str, output_type="string")
last_date_of_last_month = DsnpHelperFunction.last_date_of_last_month(today_str)
reporting_year_value=last_date_of_last_month.year
print(reporting_year_value)
reporting_end_date = last_date_of_last_month_str
print(reporting_end_date)

In [ ]:
# set the storage and output path
storage_path = r'//mbip/medicarepBI/Projects/COE/DSNP/dsnp_data_storage/monthly_flat_files'
output_path = f'{storage_path}/{reporting_end_date}'

isExist = os.path.exists(output_path)
print(output_path)
if not isExist:
    # Create a new directory because it does not exist
     os.makedirs(output_path)
     print("The new directory is created!")
else:
      print("The folder already exist")
print("Output files will sit in the folder above.")

In [ ]:
# locate data dump folder

if new_pull:
    pull_member_phone_info = snpQueries.pull_member_phone_info()
    pull_member_do_not_mail = snpQueries.pull_member_do_not_mail()
    pull_member_do_not_mail["ok_to_mail_ind"] = np.where(pull_member_do_not_mail["DirectMailPermInd"] == 'Y', 1, 0)

    pull_member_phone_info.to_parquet(f"{output_path}/pull_member_phone_info.parquet")
    pull_member_do_not_mail.to_parquet(f"{output_path}/pull_member_do_not_mail.parquet")
else:
    try:
        pull_member_phone_info = pd.read_parquet(f"{output_path}/pull_member_phone_info.parquet")
        pull_member_do_not_mail = pd.read_parquet(f"{output_path}/pull_member_do_not_mail.parquet")

    except:
        pull_member_phone_info = snpQueries.pull_member_phone_info()
        pull_member_do_not_mail = snpQueries.pull_member_do_not_mail()
        pull_member_do_not_mail["ok_to_mail_ind"] = np.where(pull_member_do_not_mail["DirectMailPermInd"] == 'Y', 1, 0)

        pull_member_phone_info.to_parquet(f"{output_path}/pull_member_phone_info.parquet")
        pull_member_do_not_mail.to_parquet(f"{output_path}/pull_member_do_not_mail.parquet")

In [ ]:
#dnc registry
dnc_registry_df = pd.read_csv('http://aetnet.aetna.com/dnc/documents/ATN.zip', compression='zip', header=0, sep='|', 
                usecols=['PHONE_NUMBER']).drop_duplicates().reset_index(drop=True)

In [ ]:
pull_member_phone_info["PhoneNumber"] = pull_member_phone_info["PhoneNumber"].astype(str)
dnc_registry_list = dnc_registry_df["PHONE_NUMBER"].astype(str).tolist()


In [ ]:
pull_member_phone_info["final_dnc_ind"] = np.where(pull_member_phone_info["PhoneNumber"].isin(dnc_registry_list), 1,
                                                   np.where(pull_member_phone_info["DNCInd"]=="Y", 1,
                                                            np.where(pull_member_phone_info["TCPAConsentInd"]=='N', 1
                                                                     ,0))
                                                                     )

In [ ]:
member_phone_info_df = pull_member_phone_info.drop_duplicates()

In [ ]:
# output the top rows and size
print(member_phone_info_df.shape)
member_phone_info_df.head(3)

In [ ]:
member_phone_info_df.to_parquet(f"{output_path}/member_phone_info_df.parquet")

In [ ]:
# output the process time
print("Process time -- %s seconds" % (time.time() - start_time))